In [8]:
import numpy as np
import pandas as pd
import xarray as xr
from pycontrails import Flight, Aircraft
from pycontrails.datalib.ecmwf import ERA5
from pycontrails.models.emissions import Emissions
from pycontrails.core.met_var import (
    AirTemperature,
    SpecificHumidity,
)

In [9]:
start_date = "2022-03-01 00:00:00"
end_date = "2022-03-01 23:00:00"

pressure_levels = [300, 250, 200]

In [10]:
era5pl = ERA5(
    time=(start_date, end_date),
    variables=(AirTemperature, SpecificHumidity),
    pressure_levels=pressure_levels,
)

In [11]:
# download data from ERA5 (or open from cache)
met = era5pl.open_metdataset()

In [16]:
# demo synthetic flight
flight_attrs = {
    "flight_id": "test",
    # set constants along flight path
    #"true_airspeed": 226.099920796651,  # true airspeed, m/s
    "thrust": 0.22,  # thrust_setting
    "nvpm_ei_n": 1.897462e15,  # non-volatile emissions index
    "aircraft_type": "E190",
    "wingspan": 48,  # m
    "n_engine": 2,
}

In [17]:
# Example flight
df = pd.DataFrame()
df["longitude"] = np.linspace(-25, -40, 100)
df["latitude"] = np.linspace(34, 40, 100)
df["altitude"] = np.linspace(10900, 10900, 100)
df["engine_efficiency"] = np.linspace(0.34, 0.35, 100)
df["fuel_flow"] = np.linspace(2.1, 2.4, 100)  # kg/s
df["aircraft_mass"] = np.linspace(154445, 154345, 100)  # kg
df["time"] = pd.date_range("2022-03-01T00:15:00", "2022-03-01T02:30:00", periods=100)
df["true_airspeed"] = np.linspace(230, 230, 100)  # m/s

In [18]:
flight = Flight(df, attrs=flight_attrs)

In [19]:
flight

Flight [8 keys x 100 length, 7 attributes]
	Keys: longitude, latitude, altitude, engine_efficiency, fuel_flow, ..., time
	Attributes:
	time                [2022-03-01 00:15:00, 2022-03-01 02:30:00]
	longitude           [-40.0, -25.0]
	latitude            [34.0, 40.0]
	altitude            [10900.0, 10900.0]
	flight_id           test
	thrust              0.22
	nvpm_ei_n           1897462000000000.0
	aircraft_type       E190
	wingspan            48
	n_engine            2
	crs                 EPSG:4326

In [23]:
# Intersect with met data
flight["AirTemperature"] = flight.intersect_met(met["air_temperature"], method="nearest")

In [24]:
flight["SpecificHumidity"] = flight.intersect_met(met["specific_humidity"], method="nearest")

In [26]:
flight.attrs["engine_uid"] = "CFM56-2-C5"


In [27]:
flight

Flight [10 keys x 100 length, 8 attributes]
	Keys: longitude, latitude, altitude, engine_efficiency, fuel_flow, ..., SpecificHumidity
	Attributes:
	time                [2022-03-01 00:15:00, 2022-03-01 02:30:00]
	longitude           [-40.0, -25.0]
	latitude            [34.0, 40.0]
	altitude            [10900.0, 10900.0]
	flight_id           test
	thrust              0.22
	nvpm_ei_n           1897462000000000.0
	aircraft_type       E190
	wingspan            48
	n_engine            2
	crs                 EPSG:4326
	engine_uid          CFM56-2-C5

In [28]:
emi = Emissions(met=met)

/home/ktait98/miniconda3/envs/traffic/lib/python3.10/site-packages/pycontrails/core/models.py:189: UserWarning: Met data appears to have originated from ECMWF and no humidity scaling is enabled. For ECMWF data, consider using one of 'ConstantHumidityScaling', 'ExponentialBoostHumidityScaling', or 'ExponentialBoostLatitudeCorrectionHumidityScaling'. For example: 
>>> from pycontrails.models import ConstantHumidityScaling
>>> Emissions(met=met, ..., humidity_scaling=ConstantHumidityScaling(rhi_adj=0.99))
  warnings.warn(


In [29]:
flight_emi = emi.eval(source=flight)

KeyError: "Flight instance does not contain data or attr 'true_airspeed'"